In [4]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
# load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\basel\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
# define the input text
text = "This is a long text. It has many sentences. Some of them are important, and some are not."

# tokenize the input text into sentences
sentences = text.split('. ')

# compute the embedding of each sentence using GPT-2
embeddings = []
for sentence in sentences:
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    with torch.no_grad():
        output = model(input_ids)[0][:, -1, :]
        embeddings.append(output)

In [7]:


# compute the similarity between each sentence embedding and the embedding of the entire text
text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0), text_embedding)

# rank the sentences based on their similarity score
sentence_scores = list(zip(sentences, similarity_scores))
sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

# select the top N sentences to form the summary
N = 2
summary = [s[0] for s in sentence_scores[:N]]